**Deep Learning Image Classification using Neural Network**
<br> by 
Matan-Ben Nagar
&
Yaara Kresner-Barak

## Importing our libraries for the project

In [1]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras
from keras.layers import Dense, Flatten,Conv2D,MaxPool2D,Dropout,AveragePooling2D
tfds.disable_progress_bar()

## Load rock_paper_scissors dataset
The dataset contains 2892 images of hands playing rock, paper, scissor game.
Its have two features- image (300, 300, 3) and lable.




In [2]:
##import the dataset from tensorflow_datasets library 
builder = tfds.builder('rock_paper_scissors')

Split Rock, Paper, Scissors data
The train set contains 2520 images, and the test set contains 372 images.

In [3]:
#load the train and test sets from the DB 
ds_train = tfds.load(name="rock_paper_scissors", split="train")
ds_test = tfds.load(name="rock_paper_scissors", split="test")

Shuffling and writing examples to /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incomplete6PAHF7/rock_paper_scissors-train.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incomplete6PAHF7/rock_paper_scissors-test.tfrecord
Dataset rock_paper_scissors downloaded and prepared to /root/tensorflow_datasets/rock_paper_scissors/3.0.0. Subsequent calls will reuse this data.


Converting the tensorflow dataset format into numpy format,

Create numpy arrays that contains the images and the labels separately,

And change the images three color channels RGB format to one color channel (to reduce the unimportant data)


In [4]:
train_images = np.array([example['image'].numpy()[:,:,0] for example in ds_train])
train_labels = np.array([example['label'].numpy() for example in ds_train])

test_images = np.array([example['image'].numpy()[:,:,0] for example in ds_test])
test_labels = np.array([example['label'].numpy() for example in ds_test])

Reshaping the images to 300 x 300 x 1 (add color feature- grayscale images). 

In [5]:
train_images = train_images.reshape(2520, 300, 300, 1)
test_images = test_images.reshape(372, 300, 300, 1)

getting us ready to be able to convert it from a scale of 0 to 1
instead of 0 to 255


In [6]:
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')

##Normalize the Data
Train images dividing equal by 255,
So the max value we can have is 255 because RGB values are between 0 and 255 so by doing this we're scaling every value to be between 0 & 1 and this is just a good common practice that helps you classify it.
It helps the basically network learn better than if you use the 0 to 255 values you could leave it 0 to 255 but it's just ultimately it's gonna probably decrease your performance a bit, so it's a common step to normalize between 0 & 1.


In [7]:
train_images /= 255
test_images /= 255

#Logistic regression


In [8]:
# Output layer.
model_lr = keras.Sequential([
   keras.layers.Flatten(),
   keras.layers.Dense(3, activation='softmax')
])


# adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
rmsprop_optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)

model_lr.compile(
    optimizer=rmsprop_optimizer,
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy']
)
model_lr.fit(train_images, train_labels, epochs=5, batch_size=32)


Epoch 1/5
79/79 [==============================] - 3s 9ms/step - loss: 54.2391 - accuracy: 0.3706
Epoch 2/5
79/79 [==============================] - 1s 9ms/step - loss: 41.1307 - accuracy: 0.3873
Epoch 3/5
79/79 [==============================] - 1s 9ms/step - loss: 33.5964 - accuracy: 0.4583
Epoch 4/5
79/79 [==============================] - 1s 9ms/step - loss: 28.1765 - accuracy: 0.5083
Epoch 5/5
79/79 [==============================] - 1s 9ms/step - loss: 26.2074 - accuracy: 0.5464


In [9]:
model_lr.evaluate(test_images, test_labels)



12/12 [==============================] - 0s 8ms/step - loss: 49.3277 - accuracy: 0.3414


[49.32766342163086, 0.34139785170555115]

##MLP 

This model network layer transform the 300 by 300 image into single colum,
After that we have two layers of activation relu function- because the constant gradient of ReLUs results in faster learning.
Finally ,the output layer going to be the same size as the number of labels we trying to classify- we use softmax because it efficient in classification problems. 

In [10]:
model = keras.Sequential([
  Flatten(),
  Dense(512, activation='relu'),
  Dense(256, activation='relu'),
  Dense(3, activation='softmax')
])

#setup loss function
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

#fit our data to the model
model.fit(train_images, train_labels, epochs=5, batch_size=32)

Epoch 1/5
79/79 [==============================] - 3s 28ms/step - loss: 24.8398 - accuracy: 0.4405
Epoch 2/5
79/79 [==============================] - 2s 27ms/step - loss: 1.8433 - accuracy: 0.6310
Epoch 3/5
79/79 [==============================] - 2s 27ms/step - loss: 0.9661 - accuracy: 0.7579
Epoch 4/5
79/79 [==============================] - 2s 27ms/step - loss: 0.7866 - accuracy: 0.7948
Epoch 5/5
79/79 [==============================] - 2s 27ms/step - loss: 1.5869 - accuracy: 0.6917


In [11]:
model.evaluate(test_images, test_labels)

12/12 [==============================] - 0s 13ms/step - loss: 2.0530 - accuracy: 0.4758


[2.053011417388916, 0.47580644488334656]

In this case we overfitting to our data - the model not learning the train data. 
we can see it by the results- the accuracy in the train data is 0.89 and the accuracy in the test data is 0.44.

##Convolutional Neural Network
This time the first layer will be Conv2D()  because the dataset consist of 2D images. The first variable inserted in the function is basically how many times a smaller gird is passing over the image <br>
this is how big or smaller grid is so if I said three and I didn't pass in three to start off and we'll leave the rides at one two one that just means they'll move one every time so it's gonna be a sliding window of three by three 

In [14]:
pip install -U keras-tuner

     |████████████████████████████████| 98 kB 3.6 MB/s 


In [16]:
# Instructions and exmaples of how to use Keras Tuner can be found in here 
# https://keras.io/keras_tuner/
from kerastuner.tuners import RandomSearch

def build_model(hp):
  model = keras.Sequential()

  model.add(AveragePooling2D(6,3,input_shape=(300,300,1)))

  model.add(Conv2D(64,3,activation='relu'))
  model.add(Conv2D(32,3,activation='relu'))

  model.add(MaxPool2D(2,2))
  model.add(Dropout(0.5))
  model.add(Flatten())
  #hp.Choise function will allow us to test out different variables for this dense layer 
  model.add(Dense(hp.Choice("Dense layer", [64, 128, 256, 512, 1024,2048]), activation='relu'))

  model.add(keras.layers.Dense(3, activation='softmax'))

  model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
  
  return model

# randomSearch will run random combination we insert into the model until it reaches the max_trials or until all of the trials are done.
# the tuner will result in giving an array of models with different combination from the one which performed highest to the lowest
# we can get the models by using tuner.get_best_model()
tuner = RandomSearch(
    # calling the build model function
    build_model, 
    # our objective is the validation accuracy - how well will it do on our test set?
    objective='val_accuracy',
    max_trials=8,
)
# what are we trying to optimise this network for? validation_data
tuner.search(train_images, train_labels, validation_data=(test_images, test_labels), epochs=10, batch_size=32)


Trial 13 Complete [00h 00m 42s]
val_accuracy: 0.725806474685669

Best val_accuracy So Far: 0.8064516186714172
Total elapsed time: 00h 13m 57s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [17]:
tuner.get_best_models()[0].evaluate(test_images,test_labels)

12/12 [==============================] - 1s 29ms/step - loss: 0.6035 - accuracy: 0.8065


[0.6035187244415283, 0.8064516186714172]